In [1]:
pip install torch transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset


In [3]:
model_name = "t5-small"  # or "gpt2", "facebook/bart-base", etc.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
# Example using a custom dataset with style labels "formal" and "informal"
data = {
   "text": ["I would like to express my gratitude.", "Can you send me the file?"],
   "style": ["formal", "informal"]
}

# Converting it to a Hugging Face dataset format
from datasets import Dataset
dataset = Dataset.from_dict(data)

# Split into training and validation sets
dataset = dataset.train_test_split(test_size=0.2)


In [13]:
def preprocess_function(examples):
    inputs = [f"{label}: {text}" for label, text in zip(examples["style"], examples["text"])]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Use input_ids as labels, which is suitable for tasks like translation
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

# Apply preprocessing to the dataset
tokenized_data = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [9]:
training_args = TrainingArguments(
   output_dir="./style-transfer",
   evaluation_strategy="epoch",
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   num_train_epochs=3,
   save_steps=10_000,
   save_total_limit=2,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer
)

# Start training
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,15.845338
2,No log,15.854301
3,No log,15.795380


TrainOutput(global_step=3, training_loss=9.164503733317057, metrics={'train_runtime': 20.714, 'train_samples_per_second': 0.145, 'train_steps_per_second': 0.145, 'total_flos': 101506351104.0, 'train_loss': 9.164503733317057, 'epoch': 3.0})

In [15]:
def style_transfer(text, style_label):
    # Prepend the style label as a prompt
    input_text = f"{style_label}: {text}"
    inputs = tokenizer(input_text, return_tensors="pt").input_ids
    output = model.generate(inputs)  # Only pass input_ids here
    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result

# Test the function
text = "Could you help me with this task?"
style_label = "formal"
print("Style Transfer Result:", style_transfer(text, style_label))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Style Transfer Result: Formal: Could you help me with this task?


In [16]:
model.save_pretrained("./style-transfer-model")
tokenizer.save_pretrained("./style-transfer-tokenizer")


('./style-transfer-tokenizer/tokenizer_config.json',
 './style-transfer-tokenizer/special_tokens_map.json',
 './style-transfer-tokenizer/spiece.model',
 './style-transfer-tokenizer/added_tokens.json',
 './style-transfer-tokenizer/tokenizer.json')

In [17]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("./style-transfer-model")
tokenizer = AutoTokenizer.from_pretrained("./style-transfer-tokenizer")


In [12]:
# Text Style Transfer in Google Colab

# Step 1: Install Necessary Libraries
# !pip install torch transformers datasets

# Step 2: Import Libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

# Step 3: Set Up Pretrained Model
model_name = "t5-small"  # or "gpt2", "facebook/bart-base", etc.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Step 4: Prepare Your Dataset
# Here we create a simple example dataset with style labels "formal" and "informal"
data = {
    "text": ["I would like to express my gratitude.", "Can you send me the file?"],
    "style": ["formal", "informal"]
}

# Converting the data to a Hugging Face Dataset format
dataset = Dataset.from_dict(data)

# Split into training and validation sets
dataset = dataset.train_test_split(test_size=0.2)

# Step 5: Preprocess the Data
def preprocess_function(examples):
    # Prepend the style label to the text as a prompt
    inputs = [f"{label}: {text}" for label, text in zip(examples["style"], examples["text"])]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    return model_inputs

# Apply preprocessing to the dataset
tokenized_data = dataset.map(preprocess_function, batched=True)

# Step 6: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./style-transfer",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
)

# Step 7: Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer
)

# Start training
trainer.train()

# Step 8: Test the Model on New Inputs
def style_transfer(text, style_label):
    input_text = f"{style_label}: {text}"
    inputs = tokenizer(input_text, return_tensors="pt")
    output = model.generate(**inputs)
    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result

# Example usage
text = "Could you help me with this task?"
style_label = "formal"
print("Style Transfer Result:", style_transfer(text, style_label))

# Step 9: Save the Model
model.save_pretrained("./style-transfer-model")
tokenizer.save_pretrained("./style-transfer-tokenizer")

# Optional: Load the Model Later
# model = AutoModelForSeq2SeqLM.from_pretrained("./style-transfer-model")
# tokenizer = AutoTokenizer.from_pretrained("./style-transfer-tokenizer")


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds